In [4]:
import chardet
import pandas as pd

with open('/Users/shaleentaneja/Downloads/Senior Data Engineer/phrases.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


phrases = pd.read_csv('/Users/shaleentaneja/Downloads/Senior Data Engineer/phrases.csv', encoding=result['encoding'])

In [35]:
phrases.iloc[4]

Phrases    what are the total losses for companies in cou...
Name: 4, dtype: object

In [22]:
!pip install gensim nltk 

In [8]:
import gensim
from gensim.models import KeyedVectors

/Users/shaleentaneja/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [127]:
wv = KeyedVectors.load_word2vec_format("/Users/shaleentaneja/Downloads/GoogleNews-vectors-negative300.bin.gz", binary=True, limit=1000000) 

In [128]:
wv.save('vectors.pickle')

In [129]:
wv = KeyedVectors.load('vectors.pickle')

In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shaleentaneja/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

In [74]:
import re
import numpy as np
from operator import add 

def wv_mapping(row, wv):
    row_mapping = []
    for word in row.split():
        word = re.sub('[^A-Za-z0-9]+', '', word)
        try:
            row_mapping.append(wv.get_vector(word.lower()))
        except:
            row_mapping.append([0] * wv.vector_size)
    return row_mapping

def normalise(row, wv):
    normalised_row = [0] * wv.vector_size
    for word_mapping in row:
        normalised_row = list(map(add, normalised_row, word_mapping))
    norm = np.linalg.norm(normalised_row)
    normal_array = normalised_row/norm
    return normal_array

In [84]:
phrases_normalised = phrases['Phrases'].apply(wv_mapping, wv=wv).apply(normalise, wv=wv)

In [88]:
from scipy.spatial.distance import cosine

In [99]:
cosine_matrix = pd.DataFrame()
for index, row in phrases_normalised.iteritems():
    similarities = []
    for rem_row in phrases_normalised:
        similarities.append(1 - cosine(row, rem_row))
    cosine_matrix[index] = similarities

In [115]:
def similar_phrase(phrase):
    wv_value = pd.Series(phrase).apply(wv_mapping, wv=wv).apply(normalise, wv=wv)
    max_simlarity = 0
    similiar_row = ''
    for index, row in phrases_normalised.iteritems():
        similarity = 1 - cosine(row, wv_value[0])
        if similarity > max_simlarity:
            max_simlarity = similarity
            similiar_row = phrases.iloc[index]
    return similiar_row, max_simlarity

In [122]:
similar_phrase('acquisitions in 2020')

(Phrases    What are the biggest acquisitions in 2020?
 Name: 47, dtype: object,
 0.695068245280913)